
# ES5100 — Métodos Matemáticos para Engenharia  
## Lista 4 — Zeros de Funções e Sistemas Não Lineares (Notebook em Julia)

**Objetivo.** Usar a biblioteca criada (módulo `MME`) para resolver os exercícios da **Lista 4**:
- Métodos: bisseção, ponto fixo (Picard), Newton, secante e Newton-Raphson para sistemas.
- Comparar velocidades de convergência quando solicitado e apresentar resultados com **6 algarismos significativos**.

> Referências da disciplina e da lista: material de “Zero de funções e sistemas não-lineares” e enunciados da Lista 4.



### Preparação do ambiente

A célula abaixo tenta carregar a biblioteca `MME`.  
Se o pacote não estiver disponível, definimos **funções fallback** mínimas compatíveis com a interface usada neste notebook.


In [ ]:

# Ambiente básico
using Printf
using LinearAlgebra
const TOL_DEFAULT = 1e-12
const MAXIT_DEFAULT = 100

# Tenta usar a biblioteca criada
const _has_MME = try
    @eval using MME  # ajuste aqui caso seu módulo tenha outro nome
    true
catch
    @warn "Pacote `MME` não encontrado. Usando funções fallback locais."
    false
end

# -----------------------------
# Fallbacks (somente se MME não estiver disponível)
# -----------------------------
if !_has_MME
    struct IterLog{T}
        k::Int
        x::T
        fx::T
        err::T
    end

    function newton_fallback(f, df, x0; tol=TOL_DEFAULT, maxiter=MAXIT_DEFAULT, log=false)
        x = x0
        logs = IterLog{Float64}[]
        for k in 1:maxiter
            fx = f(x)
            dfx = df(x)
            Δ = fx/dfx
            xnew = x - Δ
            err = abs(xnew - x)
            if log
                push!(logs, IterLog(k, xnew, f(xnew), err))
            end
            x = xnew
            if err < tol
                return x, logs
            end
        end
        return x, logs
    end

    function bisection_fallback(f, a, b; tol=TOL_DEFAULT, maxiter=MAXIT_DEFAULT, log=false)
        fa, fb = f(a), f(b)
        if fa*fb > 0
            error("Bisseção requer f(a)*f(b) < 0")
        end
        logs = IterLog{Float64}[]
        left, right = a, b
        for k in 1:maxiter
            mid = (left + right)/2
            fm = f(mid)
            err = (right - left)/2
            if log; push!(logs, IterLog(k, mid, fm, err)); end
            if abs(fm) < tol || err < tol
                return mid, logs
            end
            if fa*fm < 0
                right = mid
                fb = fm
            else
                left = mid
                fa = fm
            end
        end
        return (left+right)/2, logs
    end

    function fixed_point_fallback(g, x0; tol=TOL_DEFAULT, maxiter=MAXIT_DEFAULT, log=false)
        x = x0
        logs = IterLog{Float64}[]
        for k in 1:maxiter
            xnew = g(x)
            err = abs(xnew - x)
            if log; push!(logs, IterLog(k, xnew, NaN, err)); end
            x = xnew
            if err < tol
                return x, logs
            end
        end
        return x, logs
    end

    function secant_fallback(f, x0, x1; tol=TOL_DEFAULT, maxiter=MAXIT_DEFAULT, log=false)
        logs = IterLog{Float64}[]
        x_prev, x = x0, x1
        f_prev, f_curr = f(x_prev), f(x)
        for k in 1:maxiter
            denom = f_curr - f_prev
            if denom == 0
                error("Denominador zero na secante")
            end
            xnew = x - f_curr*(x - x_prev)/denom
            err = abs(xnew - x)
            if log; push!(logs, IterLog(k, xnew, f(xnew), err)); end
            x_prev, x = x, xnew
            f_prev, f_curr = f_curr, f(x)
            if err < tol
                return x, logs
            end
        end
        return x, logs
    end

    function newton_system_fallback(F, J, x0::AbstractVector; tol=TOL_DEFAULT, maxiter=MAXIT_DEFAULT, log=false)
        x = copy(x0)
        for k in 1:maxiter
            Fx = F(x)
            if norm(Fx, Inf) < tol
                return x, k
            end
            Jx = J(x)
            Δ = -Jx \ Fx
            x .= x .+ Δ
            if norm(Δ, Inf) < tol
                return x, k
            end
        end
        return x, maxiter
    end

    # Wrappers com nomes da "biblioteca"
    const MME = Module(:MME)
    @eval MME begin
        export newton, bisection, fixed_point, secant, newton_system
    end
    MME.newton = newton_fallback
    MME.bisection = bisection_fallback
    MME.fixed_point = fixed_point_fallback
    MME.secant = secant_fallback
    MME.newton_system = newton_system_fallback
end

# Aliases locais para usar abaixo (se pacote existir, usam as versões do pacote; caso contrário, os fallbacks)
const newton        = MME.newton
const bisection     = MME.bisection
const fixed_point   = MME.fixed_point
const secant        = MME.secant
const newton_system = MME.newton_system

println("Ambiente pronto. Usando biblioteca MME = ", _has_MME)



## Exercício 4–1 — Newton vs. Bisseção vs. Ponto Fixo (Picard)

Resolver a raiz positiva com 6 algarismos significativos de  
\[
f(x) = e^{-x} - 2\sqrt{x}, \quad x\in[0,1].
\]
1) Explique por que **não** é conveniente usar \(x^{(0)}=0\) (derivada envolve \(1/\sqrt{x}\)).  
2) Compare a velocidade de convergência de Newton e Bisseção.  
3) Refaça por um **ponto fixo asseguradamente convergente**: escolha \(g(x) = \frac{1}{4}e^{-2x}\), pois \(|g'(x)|=\frac{1}{2}e^{-2x}\le \tfrac{1}{2}<1\) em \([0,\infty)\), garantindo contração.


In [ ]:

# Definição do problema
f(x)  = exp(-x) - 2*sqrt(x)
df(x) = -exp(-x) - (2)*(1/(2*sqrt(x)))  # -e^{-x} - 1/sqrt(x)

# Observação: x0 = 0 é ruim pois df(0) é singular (1/sqrt(x) → ∞).
println("Observação: x0=0 é inadequado para Newton pois df(x) tem termo 1/sqrt(x).")

# 1) Newton
x0_N = 0.5
rootN, logN = newton(f, df, x0_N; tol=1e-13, maxiter=100, log=true)
@printf("Newton: raiz ≈ %.10f (|f|=%.3e) em %d iterações\n", rootN, abs(f(rootN)), length(logN))

# 2) Bisseção (intervalo [0,1], f(0)= -2*0 = 0? Cuidado: f(0)= exp(0) - 0 = 1 > 0; f(1)= e^{-1}-2 <0)
a, b = 0.0, 1.0
rootB, logB = bisection(f, a, b; tol=1e-12, maxiter=100, log=true)
@printf("Bisseção: raiz ≈ %.10f (|f|=%.3e) em %d iterações\n", rootB, abs(f(rootB)), length(logB))

# 3) Ponto fixo com contração garantida: x = g(x) = 1/4 * exp(-2x)
g(x) = 0.25*exp(-2x)
x0_G = 0.3
rootG, logG = fixed_point(g, x0_G; tol=1e-13, maxiter=200, log=true)
@printf("Ponto Fixo: raiz ≈ %.10f (|f|=%.3e) em %d iterações\n", rootG, abs(f(rootG)), length(logG))

# Comparativo simples de iterações
@printf("\nComparativo de iterações: Newton=%d, Bisseção=%d, Ponto Fixo=%d\n",
        length(logN), length(logB), length(logG))



## Exercício 4–2 — Fator de atrito em tubo liso (escoamento turbulento)

Resolver para \(f>0\) a equação
\[
\sqrt{f} = f\,[\,1.74\,\ln(\mathrm{Re}\sqrt{f}) - 0.4\,], \quad \mathrm{Re}=5000,
\]
via **Newton**, com chute inicial pela equação de **Blasius** \( f = 0.316\,\mathrm{Re}^{-0.25}\).  
Depois, verificar a (in)adequação de aplicar **substituições sucessivas** isolando \(f\) no lado direito.


In [ ]:

Re = 5000.0
f0_blasius = 0.316 * Re^(-0.25)
@printf("Chute de Blasius: f0 ≈ %.8f\n", f0_blasius)

φ(f) = sqrt(f) - f*(1.74*log(Re*sqrt(f)) - 0.4)
φ′(f) = (1/(2*sqrt(f))) - 1.74*(log(Re*sqrt(f)) + 0.5) + 0.4

fN, logF = newton(φ, φ′, f0_blasius; tol=1e-14, maxiter=100, log=true)
@printf("Newton: f ≈ %.10f, φ(f)=%.3e, it=%d\n", fN, φ(fN), length(logF))

# Substituições sucessivas diretas (isolar f no lado direito):
# f = sqrt(f) / (1.74*ln(Re*sqrt(f)) - 0.4)  ==> define g(f)
function g_bad(f)
    denom = 1.74*log(Re*sqrt(f)) - 0.4
    return sqrt(f)/denom
end

# Verifica derivada numérica de g no ponto solução para julgar contração (|g'(f*)|<1?)
function numderiv(h, x; eps=1e-7)
    (h(x+eps) - h(x-eps)) / (2eps)
end

gprime = numderiv(g_bad, fN)
@printf("g'(f*) ≈ %.4f  => %s\n", gprime, (abs(gprime)<1 ? "contração (converge)" : "não contração (pode divergir)"))



## Exercício 4–3 — Equação de estado de Van der Waals

\[
\big(P+\frac{a}{V^2}\big)\,(V-b) = RT \;\;\Longleftrightarrow\;\; P(V;T)=\frac{RT}{V-b}-\frac{a}{V^2}.
\]

Para cada gás da tabela, computar \(T_c=\frac{8a}{27Rb}\).  
- Para \(T>T_c\): mostrar que há única solução para qualquer \(P\).  
- Para \(T<T_c\): calcular a faixa \([P_{\min}(T),P_{\max}(T)]\) na qual existem **três soluções** \(V_1<V_2<V_3\) (obtida pelos extremos locais com \(dP/dV=0\)), e resolver um caso ilustrativo dentro e fora dessa faixa.


In [ ]:

# Constante dos gases
R = 0.082054  # L atm mol^-1 K^-1

# Tabela (a [L^2 atm mol^-2], b [L mol^-1])
gases = [
    ("CO2", 3.592,   0.04267),
    ("Anilina dimetílica", 37.490, 0.19700),
    ("He",   0.03412, 0.02370),
    ("NO",   1.340,   0.02789),
]

Tc(a,b) = 8a/(27R*b)

# P(V;T) e dP/dV
P_of_V(V, T, a, b) = R*T/(V - b) - a/(V^2)
dPdV(V, T, a, b)   = -R*T/(V - b)^2 + 2a/(V^3)

# Encontrar extremos (raízes de dP/dV) para T<Tc
function vdW_extrema(T, a, b; Vmin=b*1.0001, Vmax=200b, nscan=2000)
    # varredura para achar mudanças de sinal em dP/dV
    Vs = range(Vmin, Vmax; length=nscan)
    sgn_prev = sign(dPdV(first(Vs), T, a, b))
    roots = Float64[]
    vprev = first(Vs)
    for v in Iterators.drop(Vs,1)
        s = sign(dPdV(v, T, a, b))
        if sgn_prev == 0
            push!(roots, vprev)
        elseif s != sgn_prev
            # bisseção local
            left, right = vprev, v
            for _ in 1:80
                mid = (left+right)/2
                if dPdV(left, T, a, b)*dPdV(mid, T, a, b) <= 0
                    right = mid
                else
                    left = mid
                end
            end
            push!(roots, (left+right)/2)
        end
        sgn_prev = s
        vprev = v
    end
    sort(roots)
end

# Resolver P(V)=Pfixo encontrando até 3 raízes (bisseção em subintervalos com mudança de sinal)
function solve_V_for_P(Pfix, T, a, b; Vmin=b*1.0001, Vmax=200b, nscan=5000, tol=1e-10)
    F(V) = P_of_V(V,T,a,b) - Pfix
    Vs = range(Vmin, Vmax; length=nscan)
    vals = [F(v) for v in Vs]
    roots = Float64[]
    for i in 1:length(Vs)-1
        if vals[i]*vals[i+1] < 0
            left, right = Vs[i], Vs[i+1]
            for _ in 1:80
                mid = (left+right)/2
                if F(left)*F(mid) <= 0
                    right = mid
                else
                    left = mid
                end
            end
            push!(roots, (left+right)/2)
        end
    end
    sort(roots)
end

for (name, a, b) in gases
    Tc_ab = Tc(a,b)
    @printf("\nGás: %-20s  a=%.5f  b=%.5f  =>  Tc ≈ %.3f K\n", name, a, b, Tc_ab)

    # Caso T > Tc
    T_high = 1.10*Tc_ab
    # P(V) é monotônica: uma única solução para qualquer P (ilustramos para P=1 atm)
    Vroot_high = solve_V_for_P(1.0, T_high, a, b)
    @printf("  T=1.10 Tc: para P=1 atm, solução única V ≈ %g L/mol\n", Vroot_high[1])

    # Caso T < Tc
    T_low = 0.90*Tc_ab
    ext = vdW_extrema(T_low, a, b)
    if length(ext) == 2
        V1e, V2e = ext[1], ext[2]
        Pmax = P_of_V(V1e, T_low, a, b)  # máximo local
        Pmin = P_of_V(V2e, T_low, a, b)  # mínimo local
        if Pmin > Pmax; Pmin, Pmax = Pmax, Pmin; end
        @printf("  T=0.90 Tc: faixa de 3 soluções para P ∈ (%.5f, %.5f) atm\n", Pmin, Pmax)

        # Escolhe P no meio da faixa e resolve as 3 raízes
        Pmid = 0.5*(Pmin + Pmax)
        Vroots = solve_V_for_P(Pmid, T_low, a, b)
        if length(Vroots) == 3
            @printf("  Exemplo P=%.5f atm: V1=%.6g < V2=%.6g < V3=%.6g (L/mol)\n",
                    Pmid, Vroots[0+1], Vroots[1+1], Vroots[2+1])
        else
            @printf("  Aviso: não foram encontradas 3 raízes para P=%.5f (encontradas=%d)\n", Pmid, length(Vroots))
        end

        # Um P fora da faixa => solução única
        P_out = Pmax + 0.1  # um pouco acima de Pmax
        Vuniq = solve_V_for_P(P_out, T_low, a, b)
        @printf("  Exemplo P=%.5f atm (fora da faixa): solução única V ≈ %.6g L/mol\n", P_out, Vuniq[1])
    else
        @printf("  (Não foram detectados dois extremos para T=0.90 Tc — verifique parâmetros)\n")
    end
end



## Exercício 4–4 — Garrafa térmica (estado estacionário)

Sistema a resolver:
\[
\begin{aligned}
q_1 &= 10^{-9}\,\big[(T_0\!+\!273)^4 - (T_1\!+\!273)^4\big],\\[4pt]
q_2 &= 4\,(T_1 - T_2),\\[4pt]
q_3 &= 1.3\,(T_2 - T_3)^{4/3},
\end{aligned}
\qquad \text{com}\quad q_1=q_2=q_3, \; T_0=450^\circ C,\; T_3=25^\circ C.
\]

Usaremos **Newton-Raphson para sistemas** em \((T_1,T_2)\) impondo \(q_1-q_2=0\) e \(q_2-q_3=0\).


In [ ]:

T0 = 450.0
T3 = 25.0

q1(T1) = 1e-9 * ( (T0+273)^4 - (T1+273)^4 )
q2(T1,T2) = 4*(T1 - T2)
q3(T2) = 1.3 * (T2 - T3)^(4/3)

F(x) = begin
    T1, T2 = x
    [
        q1(T1) - q2(T1,T2);
        q2(T1,T2) - q3(T2)
    ]
end

# Jacobiano 2x2
function J(x)
    T1, T2 = x
    # Derivadas:
    # d/dT1 q1 = -1e-9 * 4*(T1+273)^3
    dq1_dT1 = -4e-9 * (T1+273)^3
    # d/dT1 q2 = 4 ; d/dT2 q2 = -4
    dq2_dT1 = 4.0
    dq2_dT2 = -4.0
    # d/dT2 q3 = 1.3*(4/3)*(T2 - T3)^(1/3)
    dq3_dT2 = 1.3*(4/3)*(T2 - T3)^(1/3)

    # F1 = q1 - q2 => [dq1_dT1 - dq2_dT1,    -dq2_dT2]
    # F2 = q2 - q3 => [dq2_dT1,              dq2_dT2 - dq3_dT2]
    return [
        dq1_dT1 - dq2_dT1   -dq2_dT2;
        dq2_dT1             dq2_dT2 - dq3_dT2
    ]
end

x0 = [200.0, 80.0]  # chute físico: T1 entre T0 e T2; T2 acima de T3
sol, it = newton_system(F, J, x0; tol=1e-12, maxiter=100, log=false)
T1, T2 = sol
@printf("Solução: T1 ≈ %.6f °C,   T2 ≈ %.6f °C   (it=%d)\n", T1, T2, it)
@printf("q1 = %.6e,  q2 = %.6e,  q3 = %.6e (W/m² em unidades coerentes do enunciado)\n",
        q1(T1), q2(T1,T2), q3(T2))



### Observações finais

- Resultados são sensíveis aos chutes iniciais nas abordagens abertas (Newton, Secante).  
- O método de ponto fixo exige **contração** (\(|g'(x)|<1\)) na vizinhança da solução.  
- Em Van der Waals, a presença de **três soluções** para \(T<T_c\) é delimitada pela faixa entre os extremos locais de \(P(V)\).

> Execute cada seção e, se desejar, ajuste tolerâncias (`tol`) e limites de iterações (`maxiter`).
